In [1]:
import requests
import json
import os

def save_history_base64(history_base64, filename):
    with open(filename, 'w') as f:
        f.write(history_base64)

def load_history_base64(filename):
    try:
        with open(filename, 'r') as f:
            return f.read()
    except FileNotFoundError:
        return None

def check_api_key_validity(api_key, url):
    url = url + "check_api_key/"
    try:
        response = requests.post(url, json={'gemini_api_key': api_key})
        response.raise_for_status()  # This will raise an exception for error status codes
        result = response.json()
        return result.get('status') == 'valid'
    except requests.RequestException as e:
        print(f"Error checking API key: {e}")
        if e.response is not None:
            print(f"Server response: {e.response.text}")
        return False

# API endpoint
#url = "https://fastapi.metaskepsis.com/talk_to_agents/"

# Initial setup
agent_number = 3
###
# Replace with your actual API key

while True:
    url="http://127.0.0.1:8000/"
    # Load the history at the beginning of each loop
    if os.path.exists('updated_history.pickle'):
        history_base64 = load_history_base64('updated_history.pickle')
    else:
        history_base64 = load_history_base64('zero_history.pickle')
    
    if history_base64 is None:
        print("Error: zero_history.pickle not found.")
        exit(1)
    
    # Get user input
    text = input("Enter your message (or 'quit' to exit): ")
    if text.lower() == 'quit':
        break

    # Check API key validity
    if not check_api_key_validity(gemini_api_key, url=url):
        print("Invalid API key. Please provide a valid key.")
        gemini_api_key = input("Enter a valid Gemini API key: ")
        continue

    # Prepare the API call
    files = {'history_file': ('history.pickle', history_base64, 'application/octet-stream')}
    data = {
        'text': text,
        'agent_number': agent_number,
        'gemini_api_key': gemini_api_key
    }

    # Make the API call
    try:
        response = requests.post(url+"talk_to_agents/", files=files, data=data)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        result = response.json()
        narration = result['narration']
        updated_history_base64 = result['updated_history']
        status = result['status']

        print(f"Narration: {narration}")
        print(f"Status: {status}")

        # Save the updated history
        save_history_base64(updated_history_base64, 'updated_history.pickle')
        print("Updated history saved to: updated_history.pickle")

    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")
        if response.status_code == 400 and "Invalid API key" in response.text:
            print("Invalid API key. Please provide a valid key.")
            gemini_api_key = input("Enter a valid Gemini API key: ")

    print("\n--- Next interaction ---\n")

print("Conversation ended.")

Error checking API key: 401 Client Error: Unauthorized for url: http://127.0.0.1:8000/check_api_key/
Server response: {"detail":"Invalid API key: 'NoneType' object has no attribute 'from_call'"}
Invalid API key. Please provide a valid key.
Error checking API key: 401 Client Error: Unauthorized for url: http://127.0.0.1:8000/check_api_key/
Server response: {"detail":"Invalid API key: \n  No API_KEY or ADC found. Please either:\n    - Set the `GOOGLE_API_KEY` environment variable.\n    - Manually pass the key with `genai.configure(api_key=my_api_key)`.\n    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information."}
Invalid API key. Please provide a valid key.
Error occurred: 422 Client Error: Unprocessable Entity for url: http://127.0.0.1:8000/talk_to_agents/

--- Next interaction ---

Narration: {"transcription": "Oh, hey. You know, just trying to process everything.  It's been a rough week.", "description": "Spoken quietly, eyes